In [40]:
import os
from pathlib import Path

while Path.cwd().name.lower() != "aiml25-exam" and "aiml25-exam" in str(Path.cwd()).lower():
    os.chdir("..")  # Move up one directory
print(f"Working directory set to: {Path.cwd()}")

Working directory set to: /Users/isimisi/Documents/GitHub/isimisi/AIML25-Exam


In [41]:
from src.yolo.yolo import Yolo
from ultralytics import YOLO
from src.utils.path import from_root
from src.llm_caller import LLMCaller
from src.mermaid_detector import MermaidDetector
import matplotlib.pyplot as plt
from src.mermaid_to_json import MermaidToJSON


plt.figure(figsize=(14, 10))

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [42]:
model = YOLO(from_root("models/yolo-trained.pt"))

yolo = Yolo(model)

In [43]:
from decouple import config

WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"
OPENAI_KEY = config("OPENAI_API_KEY")
# OPENAI_API_URL = "https://api.openai.com/v1"
# OPENAI_PROJECT = "proj_6SmPVkyXa0GChFv7TmDjB9B5"

model = LLMCaller(
    api_key=OPENAI_KEY,          
    model_id="gpt-4o",     
    params={
        "temperature": 0.7,
        "max_tokens": 150
    })

In [44]:
mermaid_to_json = MermaidToJSON(model)
detector = MermaidDetector(model, yolo, mermaid_to_json)


In [45]:
detector.initiate_image(str(from_root("datasets/test/images/1.png")), should_crop=True)


image 1/1 /Users/isimisi/Documents/GitHub/isimisi/AIML25-Exam/datasets/test/images/1.png: 448x640 14 sub-flows, 573.0ms
Speed: 1.3ms preprocess, 573.0ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
totale images: 14


In [46]:
detector.detect_nodes()

In [47]:
img = detector.graph_images[0]
print(img['nodes'])

['MainGroupTopCo["Main Group TopCo"]', 'GroupCo2["GroupCo 2"]', 'GroupCo3["GroupCo 3"]', 'Co3Limited["Co 3 Limited"]', 'GroupCo4["GroupCo 4"]', 'Co4Limited["Co 4 Limited"]', 'Co5Limited["Co 5 Limited"]', 'Facility1["Facility 1"]', 'Facility1Lenders["Facility 1 Lenders"]', 'Facility2["Facility 2"]', 'Facility2Lenders["Facility 2 Lenders"]', 'Facility3["Facility 3"]', 'Facility4["Facility 4"]']


In [48]:
detector.detect_edges()

In [49]:
for idx, img in enumerate(detector.graph_images):
    print(f"Mermaid {idx + 1}")
    print(img["mermaid"])
    print("\n\n")

Mermaid 1
graph TD
    MainGroupTopCo["Main Group TopCo"]
    GroupCo2["GroupCo 2"]
    GroupCo3["GroupCo 3"]
    Co3Limited["Co 3 Limited"]
    GroupCo4["GroupCo 4"]
    Co4Limited["Co 4 Limited"]
    Co5Limited["Co 5 Limited"]
    Facility1["Facility 1"]
    Facility1Lenders["Facility 1 Lenders"]
    Facility2["Facility 2"]
    Facility2Lenders["Facility 2 Lenders"]
    Facility3["Facility 3"]
    Facility4["Facility 4"]

    MainGroupTopCo -->|100%| GroupCo2
    MainGroupTopCo -->|100%| GroupCo3
    GroupCo3 -->|100%| Co3Limited
    MainGroupTopCo -->|100%| GroupCo4
    GroupCo4 -->|100%| Co4Limited
    GroupCo4 -->|100%| Co5Limited
    Facility1 --> Facility1Lenders
    Facility2 --> Facility2Lenders
    MainGroupTopCo --> Facility1
    MainGroupTopCo --> Facility2
    MainGroupTopCo --> Facility3
    MainGroupTopCo --> Facility4



Mermaid 2
graph TD
    Co6Limited["Co 6 Limited"]
    Co7Limited["Co 7 Limited"]
    Co8Limited["Co 8 Limited"]
    Co9Limited["Co 9 Limited"]
    Co10

In [50]:
detector.convert_edges()

In [51]:
graph = detector.get_graph()

In [52]:
from src.edge_validator import EdgeValidator

validator = EdgeValidator.from_json_file(str(from_root("datasets/test/json/1.json")), graph.edges)

In [53]:
validator.validate()

{'true_positives': ['maingrouptopco-facility3',
  'maingrouptopco-facility1',
  'maingrouptopco-groupco3',
  'groupco4-co4limited',
  'maingrouptopco-groupco2',
  'groupco4-co5limited',
  'co7limited-co8limited',
  'co6limited-co7limited',
  'bvigrouptopco-bvigroupsub1',
  'lifeinteresttrust2-maingrouptopco',
  'co2limited-co6limited',
  'groupco2-co2limited',
  'lifeinteresttrust1-bvigrouptopco',
  'maingrouptopco-groupco1',
  'maingrouptopco-groupco4',
  'bvigroupsub1-bvigroupsub2',
  'maingrouptopco-facility2',
  'lifeinteresttrust1-maingrouptopco',
  'co8limited-othersubs',
  'co7limited-co9limited',
  'maingrouptopco-facility4',
  'co9limited-co10limited'],
 'false_positives': ['topco-groupco4',
  'groupco3-co3limited',
  'co7limited-co11limited',
  'co7limited-co10limited',
  'topco-groupco3',
  'bvigroupsub2-co1limited',
  'groupco3-co11limited',
  'co11limited-partnership',
  'co3limited-co11limited',
  'co9limited-othersubs',
  'facility1-facility1lenders',
  'groupco1-co1limi